In [1]:
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.models import Sequential
from tqdm import tqdm
from scipy import io
from math import sqrt
import numpy as np
import pickle
import time
import os, sys

## functions

In [2]:
def multipl(a,b):
    sumofab=0.0
    for i in range(len(a)):
        temp=a[i]*b[i]
        sumofab+=temp
    return sumofab
 
def corrcoef(x,y):         # Pearson correlation coefficient
    n=len(x)
    sum1=sum(x)
    sum2=sum(y)
    sumofxy=multipl(x,y)
    sumofx2 = sum([pow(i,2) for i in x])
    sumofy2 = sum([pow(j,2) for j in y])
    num=sumofxy-(float(sum1)*float(sum2)/n)
    den=sqrt((sumofx2-float(sum1**2)/n)*(sumofy2-float(sum2**2)/n))
    return num/den

def read_dat(file):                      #read_dat('D:\\file.dat')
    d = pickle.load(open(file, 'rb'), encoding='latin1')
    return d

def takehalfnums(matrix):                #symmetric matrix중 절반숫자만 뽑아 한줄 array로 반환
    halfnums=[]
    length=len(matrix)
    for i in range(length):
        for j in range(i):
            halfnums.append(matrix[i][j])
    return np.array(halfnums)

# Experiment on DEAP

In [3]:
DEAP_SUBJECT = 32
DEAP_TRIAL = 40
DEAP_CHANNEL = 32

## Load DEAP dataset

In [4]:
DEAP_PATH = 'D:\\EEG_datasets\\DEAP\\data_preprocessed_python\\'

deap_raw_data = []
for i in tqdm(range(DEAP_SUBJECT)):
    d = pickle.load(open(DEAP_PATH +'s'+str(i+1).zfill(2)+'.dat', 'rb'), encoding='latin1')
    #labels = d['labels']
    deap_raw_data.append(d['data'])
deap_raw_data = np.array(deap_raw_data)

print('deap_raw_data.shape: ', deap_raw_data.shape)

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:04<00:00,  7.69it/s]


deap_raw_data.shape:  (32, 40, 40, 8064)


In [5]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,100))
for i in range(32):
    for j in range(1):
        plt.subplot(32, 1, 1*i+j+1)
        plt.plot(deap_raw_data[0,0,1*i+j, 800:800+512])
        plt.xticks(np.arange(0, 512, step=128), ["{:0<2d}".format(x) for x in np.arange(0, 512, step=128)], 
           fontsize=20
          )
plt.show()

<Figure size 2000x10000 with 32 Axes>

## Extract feature of DEAP
### Calculate `single-channel` features of DEAP

In [6]:
deap_single_channel = np.zeros((DEAP_SUBJECT,DEAP_TRIAL,DEAP_CHANNEL,10))
for i in tqdm(range(DEAP_SUBJECT)):
    for x in range(DEAP_TRIAL):
        for y in range(DEAP_CHANNEL):
            for z in range(10):
                if z<9:
                    data_cut=deap_raw_data[i][x][y][(807*z):(807*(z+1))]
                else:
                    data_cut=deap_raw_data[i][x][y][(807*z):8064]
                deap_single_channel[i][x][y][z]=np.mean(data_cut)

print('deap_single_channel.shape: ', deap_single_channel.shape)

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:02<00:00, 12.55it/s]


deap_single_channel.shape:  (32, 40, 32, 10)


### Calculate `channel-wise` features of DEAP

In [7]:
deap_channel_wise = np.zeros((DEAP_SUBJECT,DEAP_TRIAL,DEAP_CHANNEL,DEAP_CHANNEL))
for i in tqdm(range(DEAP_SUBJECT)):
    for x in range(DEAP_TRIAL):     
        deap_channel_wise[i][x] = np.corrcoef(deap_single_channel[i][x])
                
print('deap_channel_wise.shape: ', deap_channel_wise.shape)

100%|█████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 464.84it/s]


deap_channel_wise.shape:  (32, 40, 32, 32)


### Calculate `half data` of channel-wise feature in DEAP

In [8]:
deap_half_data = []                                                 #(1280,496)
deap_label = np.zeros((DEAP_SUBJECT*DEAP_TRIAL, DEAP_SUBJECT))      #(1280,32) one_hot
for video in range(DEAP_TRIAL):  # 40
    for user in range(DEAP_SUBJECT):  #32
        deap_half_data.append(takehalfnums(deap_channel_wise[user][video]))
        deap_label[DEAP_SUBJECT*video+user][user] = 1
deap_half_data = np.array(deap_half_data)

idx = np.arange(DEAP_SUBJECT*DEAP_TRIAL)
np.random.shuffle(idx)
deap_half_data = deap_half_data[idx]
deap_label = deap_label[idx]

print('deap_half_data.shape: ', deap_half_data.shape)
print('deap_label.shape: ', deap_label.shape)

deap_half_data.shape:  (1280, 496)
deap_label.shape:  (1280, 32)


### 10-fold validation / `channel-wise feature` / DEAP / MLP

In [9]:
k = 10
num_val_samples = len(deap_half_data) // k
num_epochs = 100
deap_mlp_acc = []
for i in tqdm(range(k)):
    # 검증 데이터 준비: k번째 분할
    test_data = deap_half_data[i * num_val_samples: (i + 1) * num_val_samples]
    test_label = deap_label[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [deap_half_data[:i * num_val_samples],
         deap_half_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_label = np.concatenate(
        [deap_label[:i * num_val_samples],
         deap_label[(i + 1) * num_val_samples:]],
        axis=0)

    model = Sequential()
    model.add(Dense(32, activation='softmax', input_shape=(496, )))
#     model.add(Dense(64, activation='relu', input_shape=(496, )))
#     model.add(Dense(32, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')
    model.fit(partial_train_data, partial_train_label, validation_data=(test_data, test_label), 
              epochs=num_epochs, batch_size=10, verbose=0, callbacks=[early_stop])
    # 검증 세트로 모델 평가
    test_loss, test_acc = model.evaluate(test_data, test_label, verbose=0)
    deap_mlp_acc.append(test_acc)
    
deap_mlp_acc_ = np.mean(deap_mlp_acc)
print(deap_mlp_acc_)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:21<00:00, 20.14s/it]


0.9984375


### Flatten `single-channel` feature of DEAP

In [46]:
deap_single_flatten = []                                            #(1280,320)
deap_label = np.zeros((DEAP_SUBJECT*DEAP_TRIAL, DEAP_SUBJECT))      #(1280,32) one_hot
for video in range(DEAP_TRIAL):
    for user in range(DEAP_SUBJECT):
        deap_single_flatten.append(deap_single_channel[user][video].reshape(320,))
        deap_label[DEAP_SUBJECT*video+user][user] = 1
deap_single_flatten = np.array(deap_single_flatten)

print('deap_single_flatten.shape: ', deap_single_flatten.shape)
print('deap_label.shape: ', deap_label.shape)

deap_single_flatten.shape:  (1280, 320)
deap_label.shape:  (1280, 32)


### 10-fold validation / `single-channel feature` / DEAP / MLP

In [47]:
k = 10
num_val_samples = len(deap_single_flatten) // k
num_epochs = 100
deap_mlp_acc_s = []
for i in tqdm(range(k)):
    # 검증 데이터 준비: k번째 분할
    test_data = deap_single_flatten[i * num_val_samples: (i + 1) * num_val_samples]
    test_label = deap_label[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [deap_single_flatten[:i * num_val_samples],
         deap_single_flatten[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_label = np.concatenate(
        [deap_label[:i * num_val_samples],
         deap_label[(i + 1) * num_val_samples:]],
        axis=0)

    model = Sequential()
    model.add(Dense(32, activation='softmax', input_shape=(320, )))
#     model.add(Dropout(0.5))
#     model.add(Dense(32, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=0, mode='auto')
    model.fit(partial_train_data, partial_train_label, validation_data=(test_data, test_label), 
              epochs=num_epochs, verbose=0, callbacks=[early_stop])
    # 검증 세트로 모델 평가
    test_loss, test_acc = model.evaluate(test_data, test_label, verbose=0)
    deap_mlp_acc_s.append(test_acc)
    
deap_mlp_acc_s_ = np.mean(deap_mlp_acc_s)
print(deap_mlp_acc_s_)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  3.00s/it]


0.30390626


---
---
# Experiment on MMIDB

In [3]:
MMIDB_SUBJECT = 100
MMIDB_TRIAL = 12 * 5
MMIDB_CHANNEL = 64

## Load MMIDB dataset

In [6]:
MMIDB_PATH = './eegmmidb_mat\\'

mmidb_raw_data = []
for i in tqdm(range(MMIDB_SUBJECT)):
    data = []
    for j in range(12):
        mat_file = io.loadmat(MMIDB_PATH + 'S' + str(i+1).zfill(3) + '/' + 
                              'S' + str(i+1).zfill(3) + 'R' +str(3+j).zfill(2))     # S001/S001R01.mat
        data.append(mat_file['record'][:MMIDB_CHANNEL, :9600])
        data.append(mat_file['record'][:MMIDB_CHANNEL, 160:160+9600])
        data.append(mat_file['record'][:MMIDB_CHANNEL, 320:320+9600])
        data.append(mat_file['record'][:MMIDB_CHANNEL, 480:480+9600])
        data.append(mat_file['record'][:MMIDB_CHANNEL, 540:540+9600])
    mmidb_raw_data.append(data)
mmidb_raw_data = np.array(mmidb_raw_data)

print('mmidb_raw_data.shape: ', mmidb_raw_data.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:20<00:00,  4.98it/s]


mmidb_raw_data.shape:  (100, 60, 64, 9600)


## Extract Feature of EEGMMIDB
### Calculate `single-channel` features of EEGMMIDB

In [7]:
mmidb_single_channel = np.zeros((MMIDB_SUBJECT,MMIDB_TRIAL,MMIDB_CHANNEL,10))
for i in tqdm(range(MMIDB_SUBJECT)):
    for x in range(MMIDB_TRIAL):
        for y in range(MMIDB_CHANNEL):
            for z in range(10):
                data_cut=mmidb_raw_data[i][x][y][(960*z):(960*(z+1))]
                mmidb_single_channel[i][x][y][z]=np.mean(data_cut)

print('mmidb_single_channel.shape: ', mmidb_single_channel.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.55it/s]


mmidb_single_channel.shape:  (100, 60, 64, 10)


### Calculate `channel-wise` features of EEGMMIDB

In [8]:
mmidb_channel_wise = np.zeros((MMIDB_SUBJECT,MMIDB_TRIAL,MMIDB_CHANNEL,MMIDB_CHANNEL))
for i in tqdm(range(MMIDB_SUBJECT)):
    for x in range(MMIDB_TRIAL):     
        for y in range(MMIDB_CHANNEL):          
            for z in range(MMIDB_CHANNEL):      
                a = mmidb_single_channel[i][x][y]
                b = mmidb_single_channel[i][x][z]
                mmidb_channel_wise[i][x][y][z] = corrcoef(a,b)
            
print('mmidb_channel_wise.shape: ', mmidb_channel_wise.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [08:51<00:00,  5.31s/it]


mmidb_channel_wise.shape:  (100, 60, 64, 64)


### Calculate `half data` of channel-wise feature in EEGMMIDB

In [9]:
mmidb_half_data = []                                                    #(6000,2016)
mmidb_label = np.zeros((MMIDB_SUBJECT*MMIDB_TRIAL, MMIDB_SUBJECT))      #(6000,100) one_hot
for video in range(MMIDB_TRIAL):
    for user in range(MMIDB_SUBJECT):
        mmidb_half_data.append(takehalfnums(mmidb_channel_wise[user][video]))
        mmidb_label[MMIDB_SUBJECT*video+user][user] = 1
mmidb_half_data = np.array(mmidb_half_data)

idx = np.arange(MMIDB_SUBJECT*MMIDB_TRIAL)
np.random.shuffle(idx)
mmidb_half_data = mmidb_half_data[idx]
mmidb_label = mmidb_label[idx]

print('mmidb_half_data.shape: ', mmidb_half_data.shape)
print('mmidb_label.shape: ', mmidb_label.shape)

mmidb_half_data.shape:  (6000, 2016)
mmidb_label.shape:  (6000, 100)


### 10-fold validation / `channel-wise feature` / EEGMMIDB / MLP

In [10]:
k = 10
num_val_samples = len(mmidb_half_data) // k
num_epochs = 1000
mmidb_mlp_acc = []
for i in tqdm(range(k)):
    # 검증 데이터 준비: k번째 분할
    test_data = mmidb_half_data[i * num_val_samples: (i + 1) * num_val_samples]
    test_label = mmidb_label[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [mmidb_half_data[:i * num_val_samples],
         mmidb_half_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_label = np.concatenate(
        [mmidb_label[:i * num_val_samples],
         mmidb_label[(i + 1) * num_val_samples:]],
        axis=0)

    model = Sequential()
    model.add(Dense(units=1024, input_dim=2016, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=512, input_dim=1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=100))
    model.add(Activation('softmax'))
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 

    # Training
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')
    model.fit(partial_train_data, partial_train_label, epochs=num_epochs, verbose=0, shuffle=True, 
              validation_data=(test_data, test_label), callbacks=[early_stop])

    # 검증 세트로 모델 평가
    test_loss, test_acc = model.evaluate(test_data, test_label, verbose=0)
    mmidb_mlp_acc.append(test_acc)
    
mmidb_mlp_acc_ = np.mean(mmidb_mlp_acc)
print(mmidb_mlp_acc_)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [09:21<00:00, 56.16s/it]


0.9783333


### Flatten `single-channel` feature of EEGMMIDB

In [11]:
mmidb_single_flatten = []                                                 #(6000,640)
mmidb_label_s = np.zeros((MMIDB_SUBJECT*MMIDB_TRIAL, MMIDB_SUBJECT))      #(6000,64) one_hot
for video in range(MMIDB_TRIAL):
    for user in range(MMIDB_SUBJECT):
        mmidb_single_flatten.append(mmidb_single_channel[user][video].reshape(640,))
        mmidb_label_s[MMIDB_SUBJECT*video+user][user] = 1
mmidb_single_flatten = np.array(mmidb_single_flatten)

idx = np.arange(MMIDB_SUBJECT*MMIDB_TRIAL)
np.random.shuffle(idx)
mmidb_single_flatten = mmidb_single_flatten[idx]
mmidb_label_s = mmidb_label_s[idx]

print('mmidb_single_flatten.shape: ', mmidb_single_flatten.shape)
print('mmidb_label_s.shape: ', mmidb_label_s.shape)

mmidb_single_flatten.shape:  (6000, 640)
mmidb_label_s.shape:  (6000, 100)


### 10-fold validation / `single-channel feature` / EEGMMIDB / MLP

In [12]:
k = 10
num_val_samples = len(mmidb_single_flatten) // k
num_epochs = 1000
mmidb_mlp_acc_s = []
for i in tqdm(range(k)):
    # 검증 데이터 준비: k번째 분할
    test_data = mmidb_single_flatten[i * num_val_samples: (i + 1) * num_val_samples]
    test_label = mmidb_label_s[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [mmidb_single_flatten[:i * num_val_samples],
         mmidb_single_flatten[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_label = np.concatenate(
        [mmidb_label_s[:i * num_val_samples],
         mmidb_label_s[(i + 1) * num_val_samples:]],
        axis=0)

    model = Sequential()
#     model.add(Dense(units=100, input_dim=640, activation='softmax'))
    model.add(Dense(units=256, input_dim=640, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=100, input_dim=256 ))
    model.add(Activation('softmax'))
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

    # Training
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')
    model.fit(partial_train_data, partial_train_label, epochs=num_epochs, verbose=0, shuffle=True, 
              validation_data=(test_data, test_label), callbacks=[early_stop])

    # 검증 세트로 모델 평가
    test_loss, test_acc = model.evaluate(test_data, test_label, verbose=0)
    mmidb_mlp_acc_s.append(test_acc)

mmidb_mlp_acc_s_ = np.mean(mmidb_mlp_acc_s)
print(mmidb_mlp_acc_s_)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:48<00:00,  4.89s/it]


0.10416667


# accuracy of using single-channel feature and channel-wise feature
| accuracy | single-channel feature | channel-wise feature |
|---|:---:|---:|
| EEGMMIDB | 66.5167% | 98.15% |
| DEAP     | 87.0968% | 99.8438% |

# accuracy of reduce trials
| trial | DEAP | EEGMMIDB |
|---|:---:|---:|
| 40 | 99.8438% |    -   |
| 39 | 99.9194% |    -   |
| 38 | 99.9174% |    -   |
| 37 | 99.9153% |    -   |
| 36 | 99.9130% |    -   |
| 35 |100.0000% |    -   |
| 34 |100.0000% |    -   |
| 33 |100.0000% |    -   |
| 32 |100.0000% |    -   |
| 31 |100.0000% |    -   |
| 30 | 99.8958% |    -   |
| 29 | 99.8913% |    -   |
| 28 | 99.8876% |    -   |
| 27 | 99.8837% |    -   |
| 26 |100.0000% |    -   |
| 25 | 99.8750% |    -   |
| 24 |100.0000% |    -   |
| 23 | 99.8630% |    -   |
| 22 | 99.8571% |    -   |
| 21 | 99.8507% |    -   |
| 20 | 99.8438% |    -   |
| 19 | 99.8333% |    -   |
| 18 | 99.8246% |    -   |
| 17 | 99.8148% |    -   |
| 16 | 99.8039% |    -   |
| 15 | 99.5833% |    -   |
| 14 | 99.7727% |    -   |
| 13 | 99.7561% |    -   |
| 12 | 99.7368% | 98.15% |
| 11 | 99.7143% | 97.6000% |
| 10 |100.0000% | 97.5600% |
|  9 |100.0000% | 97.2001% |
|  8 |100.0000% | 96.1750% |
|  7 |100.0000% | 95.0286% |
|  6 |100.0000% | 92.4000% |
|  5 | 99.3750% | 89.6800% |
|  4 | 98.3333% | 85.4000% |
|  3 | 96.6667% | 80.2000% |
|  2 | 94.9999% | 64.1000% |
|  1 | 00.0000% | 25.0000% |

# accuracy of reduce channels
| channels | DEAP | EEGMMIDB|
|----------|:----:|--------:|
| 64 |     -    | 98.15% |
| 32 |100.0000% | 94.5167% |
| 14 | 99.5238% | 56.0167% |
| 8  | 97.1094% | 26.0999% |
| 5  | 81.0156% | % |